In [1]:
import json
import pandas as pd
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [64]:
metadata = []
with open('scraped_metadata3/MasterMetadata3.txt','r') as f:
    for line in f:
        metadata.append(json.loads(line.strip()))
for item in metadata:
    item['categories'] = ', '.join(item['categories'].keys())
df = pd.DataFrame.from_dict(metadata)
#df.to_csv('MasterMetadata2.csv',encoding='utf-8')

In [117]:
simscores = []
with open('scraped_reviews/ElFarolitoSimilarities.csv','r') as f:
    for line in f:
        simscores.append(line)

In [118]:
simsdf = pd.DataFrame.from_csv('scraped_reviews/ElFarolitoSimilarities.csv')

In [119]:
dfm = simsdf.merge(df,on='BizName')

In [68]:
dfm[['BizName','latitude','longitude','Color','Score','categories','href']].to_csv('ElFarolitoSimilarities.csv',encoding='utf-8',index=False)

In [121]:
gjl = []
for ind, item in dfm.iterrows():
    gj = dict()
    gj['type'] = 'Feature'
    gj['geometry'] = {'type':'Point', 'coordinates':[item['longitude'],item['latitude']]}
    desc = 'Rating Differential: '+str(item['Score'])+'\nCategories: '+item['categories']
    gj['properties'] = {'title':item['BizName'], 'description' : desc, 'marker-color' : item['Color'], 
                        "marker-size": "medium"}
    gjl.append(gj)

In [122]:
with open('ElFarolitoSimilarities.geojson','w') as f:
    for line in json.dumps(gjl,indent=2, separators=(',', ': ')):
        f.write(line.encode('utf-8'))